In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Mai 20

@author: yaning
"""

import importlib
import numpy as np
import matplotlib.pyplot as plt
import itertools
import pickle
import math
import random
from tqdm import tqdm

# my own class files
import GPU.Receptors as Receptors
import GPU.Network as Network
import GPU.create_update_synapse as cus


importlib.reload(Receptors)
importlib.reload(Network)
importlib.reload(cus)

path = "/home/yaning/Documents/"
np.set_printoptions(threshold=np.inf)

In [3]:
with open(path + "fire_data_10p_8f_non_zero.pkl", "rb") as f:
    fire_data = pickle.load(f)

In [4]:
fire_data.shape

(100, 10, 10, 8, 6000)

In [5]:
pointCount = 6000
# deltaTms = 0.05
# times = np.arange(pointCount) * deltaTms
# initial_Vm = 1.3458754117369027

Input neurons

In [ ]:
pixel = 10
feature = 8
input_array = np.empty([pixel,pixel,feature], dtype=object)

for i in range(pixel):
    for j in range(pixel):
        for k in range(feature):
            input_array[i,j,k] = Network.Neuron((i,j,k))
            

# not sure if need to flatten it 
input_array_flat = input_array.flatten()

Excitatory Neurons

In [7]:
E_height = 20
E_width = 20

E_array = np.empty([E_height, E_width], dtype=object)

for i in range(E_height):
    for j in range(E_width):
        E_array[i,j] = Network.Neuron((i,j))

E_array_flat = E_array.flatten()

Inhibitory Neurons

In [8]:
I_amount = 10

I_array = np.empty(I_amount, dtype=object)
for i in range(I_amount):
    I_array[i] = Network.Neuron((i))

Ouput Neurons

In [9]:
output_size = 5

output_array = np.empty([output_size, output_size], dtype=object)

for i in range(output_size):
    for j in range(output_size):
        output_array[i,j] = Network.Neuron((i,j))
        
output_array_flat = output_array.flatten()

Connection them!

In [10]:
# Input to E (all-to-all connection) This might not be all to all connection 
# because from V1 to V2 the topological features hold
for i in input_array_flat:
    for j in E_array_flat:
        cus.create_synapse(i, j, "AMPA")

# E to I p=0.1
for i in E_array_flat:
    for j in I_array:
        if random.random() < 0.1:
            cus.create_synapse(i, j, "AMPA+NMDA")

# I to I p=0.1 (should i prevent self connection? they exist but rare)
for i in I_array:
    for j in I_array:
        if random.random() < 0.1:
            cus.create_synapse(i, j, "GABA")

# I to E p=0.024
for i in I_array:
    for j in E_array_flat:
        if random.random() < 0.024:
            cus.create_synapse(i, j, "GABA")

# # E to output (subsets)
stride = 4
kernel_size = 4

for i in output_array_flat:
    idx = i.id
    x_num = idx[0]
    y_num = idx[1]
    connect_arrary = []
    x_select = E_array[stride*x_num:stride*x_num+kernel_size]
    for j in range(len(x_select)):
        connect_arrary.append(x_select[j][stride*y_num:stride*y_num+kernel_size])
    
    connect_arrary = list(itertools.chain.from_iterable(connect_arrary))
    # connect all the selected E neurons
    for c in connect_arrary:
        cus.create_synapse(c, i, "AMPA")

In [11]:
len(cus.all_synapses)

320876

Run

In [12]:
fire_array = fire_data[:1]

In [13]:
fire_array.shape

(1, 10, 10, 8, 6000)

In [14]:
input_array.shape

(10, 10, 8)

In [ ]:
for trial in range(fire_array.shape[0]):
    fired_synapse = []
    for t in tqdm(range(pointCount)):
        this_fire_array = fire_array[trial, :, :, :, t]
        for i in range(input_array.shape[0]):
            for j in range(input_array.shape[1]):
                for k in range(input_array.shape[2]):
                    if this_fire_array[i][j][k]:
                        input_array[i][j][k].sending_signal()
                        input_array[i][j][k].fire_tstep.append(t)
            

        # update E
        for neuron in E_array_flat:
            neuron.update()
            # if fires send signal and add fire_tstep
            neuron.check_firing(t)

        for neuron in I_array:
            neuron.update()
            neuron.check_firing(t)

        for neuron in output_array_flat:
            neuron.update()
            neuron.check_firing(t)         



            # # ignore weight update for now 
            # if neuron.fire_tstep == []:
            #     last_fire = -2
            # else:
            #     last_fire = neuron.fire_tstep[-1]



            # if fire:
            #     # only update weight when it fires the first tstep
            #     if last_fire + 1 != t:
            #         weight_update_num += 1
            #         # print("this line runs")
            #         # neuron.update_weights_hebbian(t)
            #         neuron.update_weights_twenty(t)

            
            


        
        
        # set the synapse states back to 0
        for synapse in cus.all_synapses:
            if synapse.state == 1:
                # print("this line runs")
                # this shouldn't be fired_synapse but it should be activated_synapse
                if synapse not in fired_synapse:
                    fired_synapse.append(synapse)
            synapse.state = 0
        # print(fired_synapse)

    # # ignore learning as well
    # desire = dataset[trial, -1]
    # print(len(fired_synapse))

    # difference = len(neuron_output_0.fire_tstep) - len(neuron_output_1.fire_tstep)
    # if  difference > 0:
    #     real = 0
    # else: real = 1
    # print(real)
    # # # rescaled_difference = sigmoid(difference)
    # if desire == real:
    #     for synapse in fired_synapse:
    #         for receptor in synapse.receptors:
    #             learning_weight_num += 1
    #             receptor.w = receptor.w * 1.1
    # else:
    #     for synapse in fired_synapse:
    #         for receptor in synapse.receptors:
    #             receptor.w = receptor.w * 0.9
    

    # Vm_neuron_output.append(neuron_excite_main_0.Vm - 70)
    # neuron_output_currents.append(neuron_output.I)
    # neuron_excite_main_0_currents.append(neuron_excite_main_0.I)
    # neuron_excite_main_firing_tstep.append(neuron_excite_main.fire_tstep)